### Import python library

In [1]:
import numpy as np
import pandas as pd

### 1. Read data: bigcon_final_dataset.csv필요

In [2]:
data = pd.read_csv("bigcon_final_dataset.csv")
data.drop(["Unnamed: 0"],axis=1, inplace = True)

In [3]:
team = list(set(data.T_ID))

### 2. Train 전처리

In [4]:
train = data[data["IDX"]!=777]

In [5]:
sel_col = ["T_ID","IDX","MERGE_IDX","PCT","ERA","AVG",'YEAR']
train_shift = train.loc[:,sel_col]
shift_col = sorted(list(set(list(train.columns))-set(['T_ID','YEAR'])))

for i in range(1,2):
    for c in shift_col:
        train_shift.loc[:,'shift_{}'.format(c)] = train.loc[:,c].shift(i)

In [6]:
train_shift.dropna(inplace = True)
ts_idx = list(train_shift.index)

drop_idx = []
for i in ts_idx:
    idx_num = train_shift.loc[i,'IDX']
    if idx_num <= train_shift.loc[i,'shift_IDX']:
        drop_idx.append(i)

train_s_df = train_shift.drop(drop_idx).reset_index(drop = True)  

train_s_drop_df = train_s_df.drop(["IDX","MERGE_IDX","shift_IDX","shift_MERGE_IDX","ERA","AVG","PCT"],axis = 1)

In [7]:
train_s_df.shape

(2920, 66)

### 3. Test 전처리

In [8]:
test = data[data["IDX"]==777]

In [9]:
tmp1619 = data[data["IDX"]==5]
test = pd.concat([test,tmp1619],axis=0)

tmp2020 = data[data["YEAR"]==2020]
for t in team:
    tmp = tmp2020[tmp2020["T_ID"]==t]
    mer_idx = max(tmp.MERGE_IDX)
    test = pd.concat([test,tmp[tmp["MERGE_IDX"]==mer_idx]], axis = 0)
    

test = test.sort_values(by=["YEAR","T_ID","IDX"]).reset_index(drop=True)
    

In [10]:
test_shift = test.loc[:,sel_col]
shift_col = sorted(list(set(list(test.columns))-set(['T_ID','YEAR'])))

for i in range(1,2):
    for c in shift_col:
        test_shift.loc[:,'shift_{}'.format(c)] = test.loc[:,c].shift(i)

In [11]:
test_s_df = test_shift[test_shift["IDX"]==777].reset_index(drop=True)
test_s_drop_df = test_s_df.drop(["IDX","MERGE_IDX","shift_IDX","shift_MERGE_IDX","ERA","AVG","PCT"],axis = 1)

### 4.  split data  -> PCT/AVG/ERA dataset 가공

In [12]:
common_list = ['T_ID', 'YEAR', 'HEADER_NO','PCT']
hitter = ['AVG', 'OBP', 'SLG', 'OPS', 'RC', 'GPA', 'BA', 'SECA', 
          'TA', 'XR', 'ISO', 'wOBA', 'P_HRA_RT', 'DER']
pitcher = ['WHIP', 'LOB', 'ERA', 'FIP', 'H_9', 'K_9', 'BB_9', 
           'HR_9', 'oAVG', 'oSLG', 'oOBP', 'P_WHIP_RT', 'P2_WHIP_RT', 'CB_WHIP_RT','oISO', 'oOPS']

vs_hitter = ['vs_AVG', 'vs_OBP', 'vs_SLG', 'vs_OPS', 'vs_RC', 'vs_GPA', 'vs_BA', 'vs_SECA', 
             'vs_TA', 'vs_XR', 'vs_ISO', 'vs_wOBA', 'vs_P_HRA_RT', 'vs_DER']
vs_pitcher = ['vs_WHIP', 'vs_LOB', 'vs_ERA', 'vs_FIP', 'vs_H_9', 'vs_K_9', 'vs_BB_9', 
              'vs_HR_9', 'vs_oAVG', 'vs_oSLG', 'vs_oOBP', 'vs_P_WHIP_RT', 'vs_P2_WHIP_RT', 'vs_CB_WHIP_RT'
             ,'vs_oISO', 'vs_oOPS']

avg_col = common_list + hitter + vs_pitcher
era_col = common_list + pitcher + vs_hitter

In [59]:
sel_avg_lst == list(AVG_train_X.columns)

True

In [60]:
sel_era_lst == list(ERA_train_X.columns)

True

In [47]:
train_col = list(train_s_drop_df.columns)
sel_avg_lst =train_col[:2]
sel_era_lst = train_col[:2]

for i in train_col[2:]:
    for k in avg_col:
        if k in i:
            sel_avg_lst.append(i)
            break
    
for i in train_col[2:]:
    for k in era_col:
        if k in i:
            sel_era_lst.append(i)
            break


In [48]:
# 기존에서 제외된 column 목록
list(set(list(data.columns))- set(hitter+pitcher + vs_hitter+vs_pitcher + common_list))

['IDX', 'MERGE_IDX']

### 5. 최종 데이터셋 생성

In [49]:
#승률
PCT_train_X = train_s_drop_df
PCT_train_y = train_s_df[["T_ID","YEAR","PCT"]]

PCT_test_X = test_s_drop_df
PCT_test_y = test_s_df[["T_ID","YEAR","PCT"]]

In [50]:
# 타율
AVG_train_X = train_s_drop_df[sel_avg_lst]
AVG_train_y = train_s_df[["T_ID","YEAR","AVG"]]

AVG_test_X = test_s_drop_df[sel_avg_lst]
AVG_test_y = test_s_df[["T_ID","YEAR","AVG"]]

In [51]:
# 방어율
ERA_train_X = train_s_drop_df[sel_era_lst]
ERA_train_y = train_s_df[["T_ID","YEAR","ERA"]]

ERA_test_X = test_s_drop_df[sel_era_lst]
ERA_test_y = test_s_df[["T_ID","YEAR","ERA"]]

### 6. 저장

In [52]:
PCT_train_X.to_csv("PCT/PCT_train_X.csv",index=False)
PCT_train_y.to_csv("PCT/PCT_train_y.csv",index=False)

PCT_test_X.to_csv("PCT/PCT_test_X.csv",index=False)
PCT_test_y.to_csv("PCT/PCT_test_y.csv",index=False)

In [53]:
AVG_train_X.to_csv("AVG/AVG_train_X.csv",index=False)
AVG_train_y.to_csv("AVG/AVG_train_y.csv",index=False)

AVG_test_X.to_csv("AVG/AVG_test_X.csv",index=False)
AVG_test_y.to_csv("AVG/AVG_test_y.csv",index=False)

In [54]:
ERA_train_X.to_csv("ERA/ERA_train_X.csv",index=False)
ERA_train_y.to_csv("ERA/ERA_train_y.csv",index=False)

ERA_test_X.to_csv("ERA/ERA_test_X.csv",index=False)
ERA_test_y.to_csv("ERA/ERA_test_y.csv",index=False)

### <span style = "color:blue"> <참조> 팀 이름으로 one-hot 생성 </span>

In [267]:
#예시
PCT_train_X_oh = pd.concat([PCT_train_X, pd.get_dummies(PCT_train_X.T_ID)], axis = 1).drop(["T_ID"],axis=1)

In [268]:
PCT_train_X_oh

,YEAR,shift_HEADER_NO,shift_PF,shift_H_A,shift_PCT,shift_avg,shift_OBP,shift_SLG,shift_OPS,shift_RC,...,HH,HT,KT,LG,LT,NC,OB,SK,SS,WO
0,2016,0.0,972.833333,0.500000,0.500000,0.253317,0.325583,0.374167,0.699750,0.656958,...,0,0,0,1,0,0,0,0,0,0
1,2016,0.0,973.000000,0.500000,0.478261,0.291169,0.355625,0.403208,0.758833,0.825417,...,0,0,0,1,0,0,0,0,0,0
2,2016,0.0,968.625000,0.541667,0.375000,0.305842,0.363542,0.410750,0.774292,1.135625,...,0,0,0,1,0,0,0,0,0,0
3,2016,0.0,981.000000,0.416667,0.458333,0.297110,0.366333,0.432083,0.798417,1.201167,...,0,0,0,1,0,0,0,0,0,0
4,2016,0.0,972.833333,0.500000,0.500000,0.256379,0.327667,0.378792,0.706458,0.656500,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,2019,0.0,1025.375000,0.500000,0.500000,0.270784,0.324125,0.399000,0.723125,0.465083,...,0,0,0,0,0,0,0,0,1,0
2916,2019,0.0,1025.000000,0.375000,0.391304,0.269821,0.330208,0.387167,0.717375,0.432292,...,0,0,0,0,0,0,0,0,1,0
2917,2019,0.0,1024.416667,0.458333,0.416667,0.282396,0.333167,0.405708,0.738875,0.644417,...,0,0,0,0,0,0,0,0,1,0
2918,2019,0.0,1025.375000,0.500000,0.500000,0.271106,0.324375,0.391292,0.715667,0.469250,...,0,0,0,0,0,0,0,0,1,0
